In [ ]:
# Import of the libraries
import warnings
warnings.simplefilter('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os.path import join, isdir, basename, splitext, dirname, isfile
import os
from nilearn import plotting
%matplotlib inline

## Input definition

In [ ]:
caps_dir = os.path.join(os. getcwd(),'data/caps_IXI') #where output of t1-linear of clinica is saved
png_dir = os.path.join(os. getcwd(),'data/png_IXI') #where png files for the QC will be saved
part_sess = pd.read_csv(os.path.join(os. getcwd(),'data/participants_sessions_tsv.tsv'), sep='\t') #tsv with two columns: participant_id and session_id
output_file = os.path.join(os. getcwd(),'data/qc.txt') #output file 


## From the 3D T1w brain MRI in the MNI space to the PNG file


In [ ]:
png_data_location = [] #list of path to the png files

for i in range(len(part_sess)):
    participant_id = part_sess.participant_id[i]
    session_id = part_sess.session_id[i]
    im =  os.path.join(caps_dir, 'subjects', 
                      participant_id, session_id, 't1_linear', 
                       participant_id + '_' + session_id + '_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.nii.gz')
    output_png = os.path.join(png_dir, 
                    participant_id + '_' + session_id + '_T1w_space-MNI152NLin2009cSym_res-1x1x1_T1w.png')
    png_data_location.append(output_png)

    if not os.path.isfile(output_png):
        im = nib.load(im)
        im_plot = plotting.plot_anat(anat_img = im, annotate = False, 
                                     draw_cross = False, cut_coords=(-5,0,0), dim=-1)
        im_plot.savefig(output_png)

list_images = list(png_data_location)

## Definition of an alphabet for the correspondance between code assigned by the user and final label of the QC

In [ ]:
alphabet = {
            'r':[(1, 'Not SR', 'display_obv'), (4, 'Not a correct 3D T1w brain MRI', '')],
            'k':[(1, 'No Gadolinium injection', 'display_implicit'), (1, 'Gadolinium injection', '')],
            'a':[(1, 'No movement', 'display_implicit'), (2, 'Mvt 1', ''), (3, 'Mvt 2', '')],
            'z':[(1, 'Good contrast', 'display_implicit'), (2, 'Contrast 1', ''), (3, 'Contrast 2', '')],
            'd':[(1, 'No noise', 'display_implicit'), (2, 'Noise 1', ''), (3, 'Noise 2', '')]
            
           }



In [ ]:
def interpret_code(codeword, alphabet):
    curr_tier = 0
    curr_values = []
    for key in alphabet.keys():
   
        count_key = codeword.count(key)

        if count_key >= len(alphabet[key]):
            print( 'Codeword is uninterpretable')

        key_tier = alphabet[key][count_key][0]

        if key_tier > curr_tier:
            curr_tier = key_tier
       

        
        string = alphabet[key][count_key][1]
  
        if alphabet[key][count_key][2] == '':
            curr_values.append(string)
            
    if str(curr_tier) == '4':
        return "Straigh Reject"
    else:
        return "Tier {0} - ".format(curr_tier) + ', '.join(curr_values)


def get_last_annotated_image(save_file):
    """Returns the index of the last image annotated"""
    import pandas as pd
    fich = pd.read_csv(save_file, sep='|')
    return max(fich['0 '])

## Labelisation of the images

In [ ]:
# Cell handling the widgets for annotating images.

a = list_images



output2 = widgets.Output()
text = widgets.Text()

display(text, output2)

global sem, prev_size, ind_img, prev_key
sem = 0
prev_size = 0



if isfile(join(output_file)):
    ind_img = get_last_annotated_image(join(output_file))
else:
    ind_img = -1


prev_key = ''

def on_value_change(change):
    """Function called when a key is entered by an annotator. Displays the previous label, the current image to be annotated and the current annotation.
    Pass from one image to the next by entering space. Get back to the previous image by deleting the current codeword with the last inputed space.
    """
    global sem, prev_size, ind_img, prev_key

    with output2:
        display_options = ['display_optional']
        
        if sem==0:
            value = change['new']['value']
            new_size = len(value)
            
            size_has_increased = new_size > prev_size
            new_space = size_has_increased and value.endswith(' ')
            space_erased = not size_has_increased and prev_key==' '
            
            if new_space and ind_img >= 0:
                # Saves the last inputed code and its corresponding label in `name_save_file`.
                
                save_file = join(output_file)
                if os.path.isfile(save_file):
                    fich = open(save_file, 'a')
                else:
                    fich = open(save_file, 'w')
                    fich.write(' | '.join([str(0), 'img_id', 'code', 'label'])+'\n')
                img_id = list_images[ind_img].split('/')[-1]
                code = value.split(' ')[-2]
                label1 = interpret_code(code, alphabet)#, display_options=display_options)
                fich.write(' | '.join([str(ind_img), img_id, code, label1])+'\n')
                fich.close()
  
            
            if new_space or space_erased:
                # Changes the image based on the input of the annotator
                
                if new_space:
                    ind_img += 1
                else:
                    ind_img -= 1
                    
                clear_output(wait=True)
                
                prev_code = value.split(' ')[-2]
                prev_label = interpret_code(prev_code, alphabet)#, display_options=display_options)
                print("Image {0}/{1}".format(ind_img+1, len(list_images)))
                print("Previous label: {0}".format(prev_label))
                               
                print(list_images[ind_img])
                display(Image(filename=list_images[ind_img], width=1500))
                
                curr_code = value.split(' ')[-1]
                curr_label = interpret_code(curr_code, alphabet)#, display_options=display_options)
                print('\rCurrent label: {0}'.format(curr_label), end='')
                


            else:
                # Updates the current label being inputed by the annotator
                
                curr_code = value.split(' ')[-1]
                curr_label = interpret_code(curr_code, alphabet)#, display_options=display_options)
                print('\r'+curr_label, end='')
                
            prev_size = new_size
            prev_key = value[-1]
            
            sem=1
        else:
            sem+=1
        if sem==3:
            sem=0
            
text.observe(on_value_change)